# OR Advent 2024 December 12: TSP (Kotlin)


## Dependencies

This Jupyter Notebook solves an OR Advent problem with [Timefold Solver](https://timefold.ai/open-source-solver), the open source planning solver AI. We add it as a dependency:

In [27]:
@file:DependsOn("ai.timefold.solver:timefold-solver-core:1.17.0")


## Domain

We will be assigning a list of visits to a single vehicle. Each Visit has the list of all other visits ids and the distance to them.

In [28]:
import ai.timefold.solver.core.api.domain.entity.PlanningEntity
import ai.timefold.solver.core.api.domain.lookup.PlanningId

data class Visit(@PlanningId val id: Int, val distanceMap: Map<Int, Int>) {
    fun distanceTo(visit: Visit) : Int {
        return distanceMap.get(visit.id)!!;
    }
}

In [29]:
import ai.timefold.solver.core.api.domain.entity.PlanningEntity
import ai.timefold.solver.core.api.domain.lookup.PlanningId
import ai.timefold.solver.core.api.domain.variable.PlanningListVariable

@PlanningEntity
class Vehicle(@PlanningId var id: Int = 0) {

    @PlanningListVariable
    val visits: List<Visit> = ArrayList()
    
    fun totalTravelDistance() : Int {
        if(visits.isEmpty()) {
            return 0;
        }

        var totalTravel = 0;
        var previousVisit  = visits.get(0);
        for (nextVisit in visits.drop(1)) {
            totalTravel += previousVisit.distanceTo(nextVisit)
            previousVisit = nextVisit
        }
        return totalTravel;
    }
}

We gather everything in a Planning Solution

In [30]:
import ai.timefold.solver.core.api.domain.solution.PlanningEntityProperty
import ai.timefold.solver.core.api.domain.solution.PlanningScore
import ai.timefold.solver.core.api.domain.solution.PlanningSolution
import ai.timefold.solver.core.api.domain.solution.ProblemFactCollectionProperty
import ai.timefold.solver.core.api.domain.valuerange.ValueRangeProvider
import ai.timefold.solver.core.api.score.buildin.hardsoft.HardSoftScore

@PlanningSolution
data class DeliveryPlan(
    @PlanningEntityProperty
    var vehicle: Vehicle? = null,

    @ProblemFactCollectionProperty
    @ValueRangeProvider
    var visits: List<Visit>? = null,

    @PlanningScore
    var score: HardSoftScore? = null
) {
    // No-arg constructor required for Timefold
    constructor() : this(null, emptyList())
}

## Constraints

The solver takes into account hard and soft constraints:

In [31]:
import ai.timefold.solver.core.api.score.buildin.hardmediumsoftlong.HardMediumSoftLongScore
import ai.timefold.solver.core.api.score.buildin.hardsoft.HardSoftScore
import ai.timefold.solver.core.api.score.stream.*
import ai.timefold.solver.core.api.score.stream.bi.BiConstraintStream
import java.time.Duration

class OrAdventConstraintProvider : ConstraintProvider {

    override fun defineConstraints(constraintFactory: ConstraintFactory): Array<Constraint>? {
        return arrayOf(
            //soft constraints
            minimizeTraveTime(constraintFactory)
        )
    }

    private fun minimizeTraveTime(constraintFactory: ConstraintFactory): Constraint {
        return constraintFactory.forEach(Vehicle::class.java)
            .penalize(HardSoftScore.ONE_SOFT, {v -> v.totalTravelDistance()})
            .asConstraint("travelDistance")
    }
}

## Input reader

In [32]:
import java.io.File

fun readDataset(): DeliveryPlan {
    val visits = mutableListOf<Visit>()

    val input: String = File("./instance.txt").readText()
    val lines = input.lines().filter {
        it.isNotBlank() && !it.startsWith("#") && !it.startsWith("EDGE")
    }
    val firstLine : String = lines.get(0);
    val dimensions = Regex("\\d+").find(firstLine)?.value!!.toInt()

    val allDigits = Regex("\\d+").findAll(lines.drop(1).joinToString())
    val chunks = allDigits.chunked(dimensions);
    for ((index, chunk) in chunks.withIndex()) {
        val distanceMap = chunk.mapIndexed { index: Int, s: MatchResult -> index to s.value.toInt() }.toMap()
        visits.add(Visit(index, distanceMap))
    }
    return DeliveryPlan(Vehicle(), visits)
}

## Solve it

Configure and run the solver:

In [33]:
import ai.timefold.solver.core.api.solver.Solver
import ai.timefold.solver.core.api.solver.SolverFactory
import ai.timefold.solver.core.api.solver.event.BestSolutionChangedEvent
import ai.timefold.solver.core.api.solver.event.SolverEventListener
import ai.timefold.solver.core.config.solver.SolverConfig
import ai.timefold.solver.core.config.solver.termination.TerminationConfig

// The sole purpose is to keep all solutions and timing, so we can draw a graph later.
data class EventListener(val eventList : MutableList<BestSolutionChangedEvent<DeliveryPlan>> = mutableListOf()) : SolverEventListener<DeliveryPlan>{
    override fun bestSolutionChanged(event: BestSolutionChangedEvent<DeliveryPlan>) {
        println("Best solution changed ${event.newBestScore}}")
        eventList.add(event);
    }
}

val solverFactory: SolverFactory<DeliveryPlan> = SolverFactory.create(
    SolverConfig()
        .withSolutionClass(DeliveryPlan::class.java)
        .withEntityClasses(Vehicle::class.java)
        .withConstraintProviderClass(OrAdventConstraintProvider::class.java)
        
        // Stop the solver if no better solution is found for 30 seconds.
        .withTerminationConfig(TerminationConfig().withUnimprovedSecondsSpentLimit(30L)))

val problem: DeliveryPlan = readDataset()

println("Solving the problem ...")
val solver: Solver<DeliveryPlan> = solverFactory.buildSolver()
var listener = EventListener()
solver.addEventListener(listener)
val solution: DeliveryPlan = solver.solve(problem)
println("Solving finished with score (${solution.score}).")

Solving the problem ...
Best solution changed 0hard/-2837soft}
Best solution changed 0hard/-2836soft}
Best solution changed 0hard/-2834soft}
Best solution changed 0hard/-2828soft}
Best solution changed 0hard/-2826soft}
Best solution changed 0hard/-2820soft}
Best solution changed 0hard/-2819soft}
Best solution changed 0hard/-2818soft}
Best solution changed 0hard/-2817soft}
Best solution changed 0hard/-2816soft}
Best solution changed 0hard/-2815soft}
Best solution changed 0hard/-2814soft}
Best solution changed 0hard/-2813soft}
Best solution changed 0hard/-2811soft}
Best solution changed 0hard/-2809soft}
Best solution changed 0hard/-2806soft}
Best solution changed 0hard/-2805soft}
Best solution changed 0hard/-2803soft}
Best solution changed 0hard/-2802soft}
Best solution changed 0hard/-2800soft}
Best solution changed 0hard/-2799soft}
Best solution changed 0hard/-2795soft}
Best solution changed 0hard/-2794soft}
Best solution changed 0hard/-2791soft}
Best solution changed 0hard/-2790soft}
B

Result:
- unimproved spent limit 10 seconds: 2751
- unimproved spent limit 30 seconds: 2719

## Print the solution

In [34]:
val result = solution.vehicle?.visits?.map {it -> it.id}
println("Solving finished with visit order (${result}")

Solving finished with visit order ([290, 260, 410, 15, 7, 328, 429, 425, 424, 72, 74, 51, 322, 215, 109, 121, 120, 197, 140, 158, 96, 308, 160, 84, 55, 129, 127, 54, 399, 320, 404, 311, 357, 258, 102, 377, 408, 186, 183, 366, 239, 211, 48, 47, 143, 295, 42, 398, 2, 332, 154, 369, 317, 402, 201, 296, 168, 351, 100, 435, 235, 212, 268, 257, 8, 83, 270, 411, 422, 200, 385, 348, 198, 139, 220, 185, 315, 223, 338, 397, 355, 353, 195, 418, 358, 342, 283, 272, 433, 428, 381, 276, 19, 406, 389, 426, 285, 219, 400, 117, 115, 217, 89, 177, 343, 244, 232, 259, 162, 94, 378, 335, 362, 45, 349, 251, 122, 184, 254, 253, 207, 405, 210, 229, 291, 437, 231, 416, 339, 392, 388, 180, 64, 284, 1, 125, 52, 324, 206, 345, 246, 319, 189, 86, 32, 309, 261, 11, 337, 274, 373, 304, 403, 365, 12, 41, 233, 213, 77, 394, 383, 67, 267, 393, 391, 384, 28, 6, 65, 25, 38, 372, 438, 330, 314, 71, 157, 23, 167, 149, 156, 216, 95, 159, 289, 107, 282, 240, 430, 182, 37, 230, 106, 214, 40, 305, 264, 255, 221, 161, 138, 374

## Statistics

For a big dataset, a schedule visualization is often too verbose.
Let's visualize the solution through statistics:

In [35]:
%use kandy

In [36]:
import org.jetbrains.kotlinx.dataframe.api.dataFrameOf

val df = dataFrameOf(
    "bestScores" to listener.eventList.map { it.newBestScore.toLevelNumbers().get(1) }, // map soft score
    "spentTime" to listener.eventList.map { it.timeMillisSpent / 1000 }
)

plot(df) {
    line {
        x("spentTime") {
            axis.name = "Time spent in seconds"
        }
        y("bestScores") {
            axis.name = "Soft Score"
        }
    }
}

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.3.3/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="SXzA8i"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
"bestScores":[-2837.0,-2836.0,-2834.0,-2828.0,-2826.0,-2820.0,-2819.0,-2818.0,-2817.0,-2816.0,-2815.0,-2814.0,-2813.0,-2811.0,-2809.0,-2806.0,-2805.0,-2803.0,-2802.0,-2800.0,-2799.0,-2795.0,-2794.0,-2791.0,-2790.0,-2787.0,-2785.0,-2784.0,-2783.0,-2782.0,-2781.0,-2780.0,-2779.0,-2778.0,-2772.0,-2771.0,-2769.0,-2768.0,-2766.0,-2765.0,-2764.0,-2759.0,-2758.0,-2757.0,-2756.0,-2755.0,-2751.0,-2747.0,-2746.0,-2743.0,-2742.0,-2741.0,-2739.0,-2738.0,-2737.0,-2736.0,-2735.0,-2734.0,-2733.0,-2730.0,-2728.0,-2727.0,-2726.0,-2723.0,-2722.0,-2719.0],
"spentTime":[0.0,0.0,0.0,0.0,1.0,2.0,7.0,9.0,9.0,9.0,10.0,12.0,13.0,16.0,17.0,19.0,21.0,22.0,24.0,25.0,28.0,29.0,31.0,31.0,31.0,41.0,50.0,53.0,55.0,59.0,61.0,63.0,64.0,64.0,69.0,75.0,78.0,79.0,84.0,86.0,88.0,88.0,94.0,95.0,104.0,112.0,115.0,127.0,130.0,134.0,137.0,162.0,167.0,173.0,174.0,178.0,178.0,182.0,186.0,193.0,197.0,211.0,216.0,216.0,226.0,254.0]
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"name":"Time spent in seconds",
"limits":[null,null]
},{
"aesthetic":"y",
"name":"Soft Score",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"spentTime",
"y":"bestScores"
},
"stat":"identity",
"sampling":"none",
"position":"identity",
"geom":"line",
"data":{
}
}]
};
 var plotContainer = document.getElementById("SXzA8i");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 <path d="M23.31924920060519 325.5 L23.31924920060519 325.5 L23.31924920060519 322.8728813559328 L23.31924920060519 317.6186440677966 L23.31924920060519 301.8559322033898 L25.15541055498355 296.6016949152545 L26.99157190936191 280.8389830508477 L36.17237868125372 278.21186440677957 L39.84470139001044 275.5847457627124 L39.84470139001044 272.95762711864427 L39.84470139001044 270.33050847457616 L41.680862744388804 267.70338983050897 L45.35318545314552 265.07627118644086 L47.18934680752388 262.44915254237276 L52.69783087065896 257.19491525423746 L54.53399222503732 251.94067796610216 L58.20631493379405 244.05932203389875 L61.87863764255077 241.43220338983065 L63.71479899692913 236.17796610169535 L67.38712170568586 233.55084745762724 L69.22328306006422 228.29661016949194 L74.73176712319929 225.66949152542384 L76.56792847757765 215.16101694915324 L80.24025118633438 212.53389830508513 L80.24025118633438 204.65254237288173 L80.24025118633438 202.02542372881362 L98.60186473011798 194.14406779661022 L115.12731691952324 188.88983050847492 L120.63580098265832 186.2627118644068 L124.30812369141505 183.63559322033962 L131.65276910892848 181.0084745762715 L135.3250918176852 178.3813559322034 L138.99741452644193 175.7542372881362 L140.8335758808203 173.1271186440681 L140.8335758808203 170.5 L150.0143826527121 154.7372881355932 L161.03135077898227 152.110169491526 L166.53983484211736 146.8559322033907 L168.37599619649572 144.2288135593226 L177.5568029683875 138.9745762711873 L181.22912567714422 136.34745762711918 L184.90144838590095 133.72033898305108 L184.90144838590095 120.58474576271237 L195.91841651217112 117.95762711864427 L197.75457786654948 115.33050847457707 L214.28003005595474 112.70338983050897 L228.9693208909816 110.07627118644086 L234.4778049541167 99.56779661017026 L256.51174120665706 89.05932203389875 L262.02022526979215 86.43220338983065 L269.3648706873056 78.55084745762724 L274.8733547504407 75.92372881356005 L320.7773886098997 73.29661016949194 L329.9581953817915 68.04237288135664 L340.97516350806166 65.41525423728854 L342.81132486244 62.78813559322043 L350.15597027995346 60.1610169491

## Notice

This code isn't optimized for benchmarking or scaling.

To learn more about planning optimization, visit [docs.timefold.ai](https://docs.timefold.ai).